In [1]:
from sympy import init_session
from sympy import div, QQ
from sympy.parsing.latex import parse_latex
from sympy.parsing.sympy_parser import parse_expr
from IPython.display import Markdown as md
from IPython.display import display
import pandas as pd

import numpy as np
import seaborn as sns

import scipy as sc
from tabulate import tabulate
from scipy.stats import cumfreq, relfreq, norm, linregress, mode
#from sympy.stats import variance, Die, Normal, Binomial, FiniteRV, density, P, E, Normal



init_session()

a, b, c, d = symbols('a b c d', real = True)
from docparts import añadir_ejercicios, doc_ejer, escribir_preambulo, escribir_ejercicios, escribir_fin

df_ejercicios = pd.DataFrame()

fichero = 'parcial 3'
titulo = 'Parcial 3ª Evaluación'
curso = '1BC'

from sympy.stats import variance, Die, Normal, Binomial, FiniteRV, density, P, E, Normal

x,y = symbols('x, y', real=True)

%matplotlib inline


IPython console for SymPy 1.4 (Python 3.6.8-64-bit) (ground types: gmpy)

These commands were executed:
>>> from __future__ import division
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.4/



In [2]:
def analisis_discreto(texto_ejercicio, str_datos, n_ejercicio='_'):
    texto_ejercicio = texto_ejercicio + str_datos
    enunciado_latex = [r'Realiza una tabla de frecuencias',
                   r'Realiza un diagrama de barras',
                   r'Calcular los parámetros de centralización',
                   r'Calcular los parámetros de posición P70, Q1, Q3, D4',
                   r'Calcular los parámetros de dispersión'
                ]
    enunciado, solucion = [], []
    enunciado = enunciado_latex

    datos = np.loadtxt(str_datos.split())
    datos = datos.astype(int)
    #x_i, f_i, F_i, r_i  = np.unique(datos),np.bincount(datos), cumfreq(datos, numbins=len(np.unique(datos)))[0].astype(int), np.multiply(relfreq(datos, numbins=len(np.unique(datos)))[0],100)
    tabla = pd.DataFrame({'x_i':np.unique(datos), 'f_i':np.unique(datos, return_counts=True)[1], 'F_i':np.unique(datos, return_counts=True)[1].cumsum(), 'h_i':np.unique(datos, return_counts=True)[1]/len(datos), 'H_i':(np.unique(datos, return_counts=True)[1]/len(datos)).cumsum(), '%_i':np.unique(datos, return_counts=True)[1]*100/len(datos), '%A_i':(np.unique(datos, return_counts=True)[1]*100/len(datos)).cumsum()}).set_index('x_i')
    solucion.append(r"\\"+tabulate(tabla, headers="keys", tablefmt="latex"))

    d = np.diff(np.unique(datos)).min()
    left_of_first_bin = datos.min() - float(d)/2
    right_of_last_bin = datos.max() + float(d)/2
    plt.clf()
    plt.hist(datos, np.arange(left_of_first_bin, right_of_last_bin + d, d), rwidth=0.9, cumulative = False)
    plt.title("Diagrama "+n_ejercicio)
    plt.savefig("../img/"+n_ejercicio)

    solucion.append(r"\\ \resizebox{0.8\textwidth}{!}{\includegraphics[width=1\columnwidth]{%s}}" % n_ejercicio)
    
    solucion.append({"media":datos.mean(), "mediana":np.percentile(datos,50), "moda":mode(datos)})
    solucion.append({"P70":np.percentile(datos,70), "Q1":np.percentile(datos,25),"Q3":np.percentile(datos,75),"D4":np.percentile(datos,40),})
    solucion.append({"rango":np.amax(datos)-np.amin(datos), "varianza": np.var(datos), "desviación típica":sqrt(np.var(datos)), "coeficiente variación": sqrt(np.var(datos))/abs(np.mean(datos))})
    display(tabla, solucion[2] , solucion[3], solucion[4])
    return  texto_ejercicio, enunciado_latex, enunciado, solucion, tabla, n_ejercicio


def datos_cualitativa(lista):
    # Devuelve un diccionario para crear la probabilidad cuantitativa a partir del índice
    nplista = np.array(lista)
    total = nplista[:,1].astype(int).sum()
    lista_val = list(nplista[:,0])
    dic ={i:nplista[i,1].astype(int)/total for i,s in enumerate(nplista)}
    
    return dic,lista_val    

def extracciones(datos, remplazamiento=False) :
    #Devuelve diccionarios con las extracciones del primer conjunto
    diccionarios = []

    if remplazamiento is True : 
        extraccion = 0
    else :
        extraccion = 1


    diccionarios.append(datos_cualitativa(datos))

    for i in range(len(datos)):
        npdatos = np.array(datos)
        npdatos[i,1]=npdatos[i,1].astype(int) - extraccion
        diccionarios.append(datos_cualitativa(list(npdatos)))
        
    return diccionarios




## Ejercicio 1

Una asociación de consumidores ha realizado una prueba sobre la duración de unas bombillas de una conocida marca. Ha mantenido encendidas ininterrumpidamente 100 bombillas hasta que se han estropeado. Sus resultados han sido:

In [3]:
# límite inferior, superior y frecuencia
#datos = np.array([[36,42,12],[42,48,28],[48,54,44],[54,60,11],[60,66,5]])
datos = np.array([[36,42,12],[42,48,28],[48,54,45],[54,60,15]])
tabla_enunciado = pd.DataFrame(list(zip([r"$"+latex(Interval.Ropen(k[0],k[1]))+"$" for k in datos[:,0:2]],datos[:,2])), columns=['Duración','Cantidad'])
tabla_enunciado_latex = tabulate(tabla_enunciado, headers="keys", tablefmt="latex").replace('\\$','$').replace('textbackslash{}','')
md(tabla_enunciado_latex)


\begin{tabular}{rlr}
\hline
    & Duración              &   Cantidad \\
\hline
  0 & $\left[36, 42\right)$ &         12 \\
  1 & $\left[42, 48\right)$ &         28 \\
  2 & $\left[48, 54\right)$ &         45 \\
  3 & $\left[54, 60\right)$ &         15 \\
\hline
\end{tabular}

In [4]:
marca = (datos[:,1]+datos[:,0])/2
marca = marca.reshape(-1,1)
datos = np.concatenate((datos, marca), axis=1)[:,[0,1,3,2]]
datos

array([[36., 42., 39., 12.],
       [42., 48., 45., 28.],
       [48., 54., 51., 45.],
       [54., 60., 57., 15.]])

In [5]:
tabla = pd.DataFrame({r'lim_inf': datos[:,0],'lim_sup': datos[:,1],
                      'x_i': datos[:,2],'f_i':datos[:,3],'F_i':datos[:,3].cumsum(),
                     'h_i': datos[:,3]/datos[:,3].sum(), 'H_i':(datos[:,3]/datos[:,3].sum()).cumsum(),
                     'x_if_i': datos[:,2]*datos[:,3], 'x^2_if_i': datos[:,2]**2*datos[:,3]})
tabla

#totales = np.array([np.nan,np.nan,np.nan,tabla.f_i.sum(),tabla.F_i.sum()])
totales = pd.DataFrame([[np.nan,np.nan,np.nan,tabla.f_i.sum(),
                         np.nan, tabla.h_i.sum(), np.nan,
                        tabla.x_if_i.sum(),tabla['x^2_if_i'].sum()]],columns=list(tabla.columns))
totales

tabla.append(totales,ignore_index=True)

,lim_inf,lim_sup,x_i,f_i,F_i,h_i,H_i,x_if_i,x^2_if_i
0,36.0,42.0,39.0,12.0,12.0,0.12,0.12,468.0,18252.0
1,42.0,48.0,45.0,28.0,40.0,0.28,0.40,1260.0,56700.0
2,48.0,54.0,51.0,45.0,85.0,0.45,0.85,2295.0,117045.0
3,54.0,60.0,57.0,15.0,100.0,0.15,1.00,855.0,48735.0
4,NaN,NaN,NaN,100.0,NaN,1.00,NaN,4878.0,240732.0


In [6]:
md(tabulate(tabla.append(totales,ignore_index=True), headers="keys", tablefmt="latex"))

\begin{tabular}{rrrrrrrrrr}
\hline
    &   lim\_inf &   lim\_sup &   x\_i &   f\_i &   F\_i &   h\_i &    H\_i &   x\_if\_i &   x\^{}2\_if\_i \\
\hline
  0 &        36 &        42 &    39 &    12 &    12 &  0.12 &   0.12 &      468 &      18252 \\
  1 &        42 &        48 &    45 &    28 &    40 &  0.28 &   0.4  &     1260 &      56700 \\
  2 &        48 &        54 &    51 &    45 &    85 &  0.45 &   0.85 &     2295 &     117045 \\
  3 &        54 &        60 &    57 &    15 &   100 &  0.15 &   1    &      855 &      48735 \\
  4 &       nan &       nan &   nan &   100 &   nan &  1    & nan    &     4878 &     240732 \\
\hline
\end{tabular}

Determina la media, la varianza y la desviación típica:

In [7]:
media = (totales['x_if_i']/totales['f_i'])[0]
media

var = (totales['x^2_if_i']/totales['f_i']-media**2)[0]

desv_tip = sqrt(var)

{'media':media, 'varianza': var, 'desviación típica': desv_tip}

{'media': 48.78,
 'varianza': 27.83159999999998,
 'desviación típica': 5.27556632031102}

Calcula la Mediana

In [8]:
# Índice donde está el percentil

perc = 50

ind = tabla.loc[tabla.H_i<=perc/100].iloc[-1:].index[0] + 1

tabla.iloc[ind]

lim_inf         48.00
lim_sup         54.00
x_i             51.00
f_i             45.00
F_i             85.00
h_i              0.45
H_i              0.85
x_if_i        2295.00
x^2_if_i    117045.00
Name: 2, dtype: float64

In [9]:
k = perc
N = totales.f_i[0]
L = tabla.iloc[ind].lim_inf
f = tabla.iloc[ind].f_i
if ind == 0 :
    F = 0
else:
    F = tabla.iloc[ind-1].F_i
C = tabla.iloc[ind].lim_sup - tabla.iloc[ind].lim_inf

print({'k':k, 'N': N, 'L_i': L, 'f_i': f, 'F_{i-1}':F, 'C_i':C})

percentil = L + C*(k*(N/100)-F)/f

percentil


{'k': 50, 'N': 100.0, 'L_i': 48.0, 'f_i': 45.0, 'F_{i-1}': 40.0, 'C_i': 6.0}


En el anuncio de televisión el fabricante asegura que sus bombillas duran más de 1200 horas. ¿Qué porcentaje de las bombillas no cumple lo enunciado?

In [9]:
valor = 1200/24
print(valor)
ind = tabla.loc[tabla.lim_inf<=valor].iloc[-1:].index[0]

if ind == 0 :
    F = 0
else:
    F = tabla.iloc[ind-1].F_i

N = totales.f_i[0]
L = tabla.iloc[ind].lim_inf
f = tabla.iloc[ind].f_i
C = tabla.iloc[ind].lim_sup - tabla.iloc[ind].lim_inf

print({'valor':valor, 'N': N, 'L_i': L, 'f_i': f, 'F_{i-1}':F, 'C_i':C})

# 100-solve(Eq(valor,L+C*(x*N/100-F)/f))[0]

solve(Eq(valor,L+C*(x*N/100-F)/f))[0]

50.0
{'valor': 50.0, 'N': 100.0, 'L_i': 48.0, 'f_i': 45.0, 'F_{i-1}': 40.0, 'C_i': 6.0}


In [19]:
240732/100


In [20]:
_-48.78** 2


In [21]:
48.78** 2

In [11]:
# Página 98 ejercicio 6

p_total, p_1, p_2, p_uni = [100,50,40,60]
p_int = p_1 + p_2 - p_uni

texto_ejercicio = r"""En Utebo, el {}\% de sus habitantes es mayor de 60 años, el {}\% fuma y el {}\% 
fuma o es mayor de 60 años. 
Calcula la probabilidad de los siguientes sucesos:""".format(p_1, p_2, p_uni) 

n_ejercicio = 'ex21e02'

display(md("**"+texto_ejercicio+"**"))

# vectores iniciales correspondientes al texto del enunciado

dic_1, val_1 = datos_cualitativa([['S',p_1],['N',p_total-p_1]])
dic_2, val_2 = datos_cualitativa([['S',p_2],['N',p_total-p_2]])

mayor = FiniteRV('mayor',density=dic_1)
casado = FiniteRV('casado',density=dic_2)


enunciado_latex, enunciado, solucion = [],[],[]

en = r"""i) Ser mayor de 60 años y fumar \\ \
ii) No fumar""" 

enunciado_latex.append(en)
enunciado = str(en)
solucion.append(latex([nsimplify(p_int/p_total),
nsimplify(P(Eq(casado,val_2.index('N'))))]))

display(md(""+ enunciado_latex[0] +" \\rightarrow " + solucion[0] +""))
    

df_ejercicios = df_ejercicios.append(añadir_ejercicios(enunciado_latex, enunciado, solucion, texto_ejercicio, curso, titulo , n_ejercicio, '1', '2', enun_tex=False), ignore_index=True)


**En Utebo, el 50\% de sus habitantes es mayor de 60 años, el 40\% fuma y el 60\% 
fuma o es mayor de 60 años. 
Calcula la probabilidad de los siguientes sucesos:**

i) Ser mayor de 60 años y fumar \\ ii) No fumar \rightarrow \left[ \frac{3}{10}, \  \frac{3}{5}\right]

In [12]:
# Página 99 ejercicio 18

# Bayes


pr0=60
pr1=1
pr2=2
datos = [[['A',pr0],['B',100-pr0]],[['D',pr1],['ND',100-pr1]],[['D',pr2],['ND',100-pr2]]]

texto_ejercicio = r"""Dos fábricas producen el mismo televisor. La fábrica A produce el {}% de todos los televisore.
El {}% de todos los televisores producidos por la fábrica A salen defectuosos, mientras que el {}% de los
televisores producidos por la fábrica B son defectuosos. Se selecciona un televisor al azar de entre
todos los fabricados:
""".format(pr0,pr1,pr2)

n_ejercicio = 'ex21e03'

display(md("**"+texto_ejercicio+"**"))

diccionarios = [datos_cualitativa(datos[i]) for i,s  in enumerate(datos)]

ini = FiniteRV('ini',density=diccionarios[0][0])
si_1 = FiniteRV('si_1',density=diccionarios[1][0])
si_2 = FiniteRV('si_2',density=diccionarios[2][0])

display(density(ini).dict,density(si_1).dict,density(si_2).dict)

enunciado_latex, enunciado, solucion = [],[],[]

en = r"""i) calcular la probabilidad de que sea defectuoso
ii) Si sabemos que el televisor es defectuoso, calcula la probabilidad de que haya sido producido por la
planta A.""" 

enunciado_latex.append(en)
enunciado = str(en)

#P(Eq(frances,val_1.index('F')))

p1 = P(Eq(ini,0))*P(Eq(si_1,0))+P(Eq(ini,1))*P(Eq(si_2,0))
p2 = (P(Eq(ini,0))*P(Eq(si_1,0)))/(P(Eq(ini,0))*P(Eq(si_1,0))+P(Eq(ini,1))*P(Eq(si_2,0)))
solucion.append(latex([nsimplify(p1),
                       nsimplify(p2)
               ]))

display(md(""+ enunciado_latex[0] +" \\rightarrow " + solucion[0] +""))

df_ejercicios = df_ejercicios.append(añadir_ejercicios(enunciado_latex, enunciado, solucion, texto_ejercicio, curso, titulo , n_ejercicio, '1', '2', enun_tex=False), ignore_index=True)


**Dos fábricas producen el mismo televisor. La fábrica A produce el 60% de todos los televisore.
El 1% de todos los televisores producidos por la fábrica A salen defectuosos, mientras que el 2% de los
televisores producidos por la fábrica B son defectuosos. Se selecciona un televisor al azar de entre
todos los fabricados:
**

i) calcular la probabilidad de que sea defectuoso
ii) Si sabemos que el televisor es defectuoso, calcula la probabilidad de que haya sido producido por la
planta A. \rightarrow \left[ \frac{7}{500}, \  \frac{3}{7}\right]

In [13]:
puntuacion = '1'                          
                      

datos = [[70,5]]

# vectores iniciales correspondientes al texto del enunciado

for i, s in enumerate(datos) : 
    n_ejercicio = 'ex21e04-'+ str(i)
    
    
    texto_ejercicio = r"""Un jugador de baloncesto tiene un porcentaje de acierto en tiros de 2 del {} \%. Si tira {}
veces:\\ 
    """.format(datos[i][0],datos[i][1])
    
    display(md("**"+texto_ejercicio+"**"))
    
    p, n  = datos[i]
    X = Binomial('X',n,p/100) 

    enunciado_latex, enunciado, solucion = [],[],[]
    
    expr = r"Describe la variable del ejercicio"
    enunciado_latex.append(expr)
    en="density(X).dict"
    enunciado.append(en)
    sol = r"$"
    sol+=latex(eval(en))+r"$"
    solucion.append(sol)
    display(md(""+latex(enunciado_latex[0]) +"$  \\rightarrow $" + latex(solucion[0])))
    
    #Calcula
    
    expr = r"Calcula la probabilidad de que enceste 4"
    enunciado_latex.append(expr)
    en="P(Eq(X,4))"
    enunciado.append(en)
    sol = r"$"+latex(parse_expr(en))+"="
    #sol+=latex(eval(enunciado[0]))+r"$ y $"+latex(eval(enunciado[1]))+r"$"
    sol+=latex(eval(en))+r"$"
    solucion.append(sol)
    display(md(""+latex(enunciado_latex[1]) +"$  \\rightarrow $" + latex(solucion[1]) ))
    
    expr = r"Calcula la probabilidad de que enceste al menos 1"
    enunciado_latex.append(expr)
    en="P(X>=1)"
    enunciado.append(en)
    sol = r"$"+latex(parse_expr(en))+"="
    #sol+=latex(eval(enunciado[0]))+r"$ y $"+latex(eval(enunciado[1]))+r"$"
    sol+=latex(eval(en))+r"$"
    solucion.append(sol)
    display(md(""+latex(enunciado_latex[2]) +"$  \\rightarrow $" + latex(solucion[2]) ))

    
    expr = r"Calcula la probabilidad de que enceste más de 4"
    enunciado_latex.append(expr)
    en = "P(X>4)"
    enunciado.append(en)
    sol = r"$"+latex(parse_expr(en))+"="
    #sol+=latex(eval(enunciado[0]))+r"$ y $"+latex(eval(enunciado[1]))+r"$"
    sol+=latex(eval(en))+r"$"
    solucion.append(sol)
    display(md(""+latex(enunciado_latex[3]) +"$  \\rightarrow $" + latex(solucion[3]) ))

    
    df_ejercicios = df_ejercicios.append(añadir_ejercicios(enunciado_latex, enunciado, solucion, texto_ejercicio, curso, titulo , n_ejercicio, '5', '1', enun_tex=False, sol_tex=False), ignore_index=True)
    



**Un jugador de baloncesto tiene un porcentaje de acierto en tiros de 2 del 70 \%. Si tira 5
veces:\\ 
    **

Describe la variable del ejercicio$  \rightarrow $$\left\{ 0 : 0.00243, \  1 : 0.02835, \  2 : 0.1323, \  3 : 0.3087, \  4 : 0.36015, \  5 : 0.16807\right\}$

Calcula la probabilidad de que enceste 4$  \rightarrow $$P{\left(X = 4 \right)}=0.36015$

Calcula la probabilidad de que enceste al menos 1$  \rightarrow $$P{\left(X \geq 1 \right)}=0.99757$

Calcula la probabilidad de que enceste más de 4$  \rightarrow $$P{\left(X > 4 \right)}=0.16807$

In [14]:
puntuacion = '1'                          
                      

datos = [[10,2,12]]

# vectores iniciales correspondientes al texto del enunciado

for i, s in enumerate(datos) : 
    n_ejercicio = 'ex21e05-'+ str(i)
    
    
    texto_ejercicio = r"""La duración media de un televisor es de {} años, con una desviación típica igual a {} años. Si la
vida útil del electrodoméstico se distribuye normalmente. Halla la probabilidad de que al comprar
un televisor: 
    """.format(datos[i][0],datos[i][1])
    
    display(md("**"+texto_ejercicio+"**"))
    
    m, d, valor  = datos[i]
    X = Normal('X',m,d) 

    enunciado_latex, enunciado, solucion = [],[],[]
    
    
    #Calcula
    
    expr = r""" este dure más de {} años""".format(valor)
    enunciado_latex.append(expr)
    en="P(X>={})".format(valor)
    enunciado.append(en)
    sol = r"$"+latex(parse_expr(en))+"="
    #sol+=latex(eval(enunciado[0]))+r"$ y $"+latex(eval(enunciado[1]))+r"$"
    sol+=latex(eval(en).evalf())+r"$"
    solucion.append(sol)
    display(md(""+latex(enunciado_latex[0]) +"$  \\rightarrow $" + latex(solucion[0]) ))
    
    
    #Calcula
    
    expr = r""" este dure entre {} y {} años""".format(datos[i][0]-2, datos[i][0]+2)
    enunciado_latex.append(expr)
    en="P(And(X>={},X<={}))".format(datos[i][0]-2, datos[i][0]+2)
    enunciado.append(en)
    sol = r"$"+latex(parse_expr(en))+"="
    #sol+=latex(eval(enunciado[0]))+r"$ y $"+latex(eval(enunciado[1]))+r"$"
    sol+=latex(eval(en).evalf())+r"$"
    solucion.append(sol)
    display(md(""+latex(enunciado_latex[1]) +"$  \\rightarrow $" + latex(solucion[1]) ))

  
    df_ejercicios = df_ejercicios.append(añadir_ejercicios(enunciado_latex, enunciado, solucion, texto_ejercicio, curso, titulo , n_ejercicio, '5', '1', enun_tex=False, sol_tex=False), ignore_index=True)

  
    df_ejercicios = df_ejercicios.append(añadir_ejercicios(enunciado_latex, enunciado, solucion, texto_ejercicio, curso, titulo , n_ejercicio, '5', '1', enun_tex=False, sol_tex=False), ignore_index=True)


**La duración media de un televisor es de 10 años, con una desviación típica igual a 2 años. Si la
vida útil del electrodoméstico se distribuye normalmente. Halla la probabilidad de que al comprar
un televisor: 
    **

 este dure más de 12 años$  \rightarrow $$P{\left(X \geq 12 \right)}=0.158655253931457$

 este dure entre 8 y 12 años$  \rightarrow $$P{\left(X \geq 8 \wedge X \leq 12 \right)}=0.682689492137086$

In [15]:
df_ejercicios

,enunciado_latex,enunciado,solucion,texto,curso,titulo,n_ejercicio,dificultad,n_columnas,puntos,enun_tex,sol_tex
0,i) Ser mayor de 60 años y fumar \\ ii) No fumar,i) Ser mayor de 60 años y fumar \\ ii) No fumar,"\left[ \frac{3}{10}, \ \frac{3}{5}\right]","En Utebo, el 50\% de sus habitantes es mayor d...",1BC,Parcial 3ª Evaluación,ex21e02,1,2,1,False,True
1,i) calcular la probabilidad de que sea defectu...,i) calcular la probabilidad de que sea defectu...,"\left[ \frac{7}{500}, \ \frac{3}{7}\right]",Dos fábricas producen el mismo televisor. La f...,1BC,Parcial 3ª Evaluación,ex21e03,1,2,1,False,True
2,Describe la variable del ejercicio,density(X).dict,"$\left\{ 0 : 0.00243, \ 1 : 0.02835, \ 2 : 0...",Un jugador de baloncesto tiene un porcentaje d...,1BC,Parcial 3ª Evaluación,ex21e04-0,5,1,1,False,False
3,Calcula la probabilidad de que enceste 4,"P(Eq(X,4))",$P{\left(X = 4 \right)}=0.36015$,Un jugador de baloncesto tiene un porcentaje d...,1BC,Parcial 3ª Evaluación,ex21e04-0,5,1,1,False,False
4,Calcula la probabilidad de que enceste al menos 1,P(X>=1),$P{\left(X \geq 1 \right)}=0.99757$,Un jugador de baloncesto tiene un porcentaje d...,1BC,Parcial 3ª Evaluación,ex21e04-0,5,1,1,False,False
5,Calcula la probabilidad de que enceste más de 4,P(X>4),$P{\left(X > 4 \right)}=0.16807$,Un jugador de baloncesto tiene un porcentaje d...,1BC,Parcial 3ª Evaluación,ex21e04-0,5,1,1,False,False
6,este dure más de 12 años,P(X>=12),$P{\left(X \geq 12 \right)}=0.158655253931457$,La duración media de un televisor es de 10 año...,1BC,Parcial 3ª Evaluación,ex21e05-0,5,1,1,False,False
7,este dure entre 8 y 12 años,"P(And(X>=8,X<=12))",$P{\left(X \geq 8 \wedge X \leq 12 \right)}=0....,La duración media de un televisor es de 10 año...,1BC,Parcial 3ª Evaluación,ex21e05-0,5,1,1,False,False
8,este dure más de 12 años,P(X>=12),$P{\left(X \geq 12 \right)}=0.158655253931457$,La duración media de un televisor es de 10 año...,1BC,Parcial 3ª Evaluación,ex21e05-0,5,1,1,False,False
9,este dure entre 8 y 12 años,"P(And(X>=8,X<=12))",$P{\left(X \geq 8 \wedge X \leq 12 \right)}=0....,La duración media de un televisor es de 10 año...,1BC,Parcial 3ª Evaluación,ex21e05-0,5,1,1,False,False


In [16]:
df_ejercicios.to_csv('ejercicios_parcial3.csv', index = False)

In [17]:
df_ejercicios = pd.read_csv('ejercicios_parcial3.csv', index_col= None)

In [18]:
escribir_preambulo(fichero, titulo, 'ejercicios')
for s in df_ejercicios.groupby('n_ejercicio').count().index : 
    display(md("**Ejercicio: **" + s ))
    display(df_ejercicios[df_ejercicios.n_ejercicio == s])
    escribir_ejercicios(df_ejercicios[df_ejercicios.n_ejercicio == s],fichero)

escribir_fin(fichero)

**Ejercicio: **ex21e02

,enunciado_latex,enunciado,solucion,texto,curso,titulo,n_ejercicio,dificultad,n_columnas,puntos,enun_tex,sol_tex
0,i) Ser mayor de 60 años y fumar \\ ii) No fumar,i) Ser mayor de 60 años y fumar \\ ii) No fumar,"\left[ \frac{3}{10}, \ \frac{3}{5}\right]","En Utebo, el 50\% de sus habitantes es mayor d...",1BC,Parcial 3ª Evaluación,ex21e02,1,2,1,False,True


**Ejercicio: **ex21e03

,enunciado_latex,enunciado,solucion,texto,curso,titulo,n_ejercicio,dificultad,n_columnas,puntos,enun_tex,sol_tex
1,i) calcular la probabilidad de que sea defectu...,i) calcular la probabilidad de que sea defectu...,"\left[ \frac{7}{500}, \ \frac{3}{7}\right]",Dos fábricas producen el mismo televisor. La f...,1BC,Parcial 3ª Evaluación,ex21e03,1,2,1,False,True


**Ejercicio: **ex21e04-0

,enunciado_latex,enunciado,solucion,texto,curso,titulo,n_ejercicio,dificultad,n_columnas,puntos,enun_tex,sol_tex
2,Describe la variable del ejercicio,density(X).dict,"$\left\{ 0 : 0.00243, \ 1 : 0.02835, \ 2 : 0...",Un jugador de baloncesto tiene un porcentaje d...,1BC,Parcial 3ª Evaluación,ex21e04-0,5,1,1,False,False
3,Calcula la probabilidad de que enceste 4,"P(Eq(X,4))",$P{\left(X = 4 \right)}=0.36015$,Un jugador de baloncesto tiene un porcentaje d...,1BC,Parcial 3ª Evaluación,ex21e04-0,5,1,1,False,False
4,Calcula la probabilidad de que enceste al menos 1,P(X>=1),$P{\left(X \geq 1 \right)}=0.99757$,Un jugador de baloncesto tiene un porcentaje d...,1BC,Parcial 3ª Evaluación,ex21e04-0,5,1,1,False,False
5,Calcula la probabilidad de que enceste más de 4,P(X>4),$P{\left(X > 4 \right)}=0.16807$,Un jugador de baloncesto tiene un porcentaje d...,1BC,Parcial 3ª Evaluación,ex21e04-0,5,1,1,False,False


**Ejercicio: **ex21e05-0

,enunciado_latex,enunciado,solucion,texto,curso,titulo,n_ejercicio,dificultad,n_columnas,puntos,enun_tex,sol_tex
6,este dure más de 12 años,P(X>=12),$P{\left(X \geq 12 \right)}=0.158655253931457$,La duración media de un televisor es de 10 año...,1BC,Parcial 3ª Evaluación,ex21e05-0,5,1,1,False,False
7,este dure entre 8 y 12 años,"P(And(X>=8,X<=12))",$P{\left(X \geq 8 \wedge X \leq 12 \right)}=0....,La duración media de un televisor es de 10 año...,1BC,Parcial 3ª Evaluación,ex21e05-0,5,1,1,False,False
8,este dure más de 12 años,P(X>=12),$P{\left(X \geq 12 \right)}=0.158655253931457$,La duración media de un televisor es de 10 año...,1BC,Parcial 3ª Evaluación,ex21e05-0,5,1,1,False,False
9,este dure entre 8 y 12 años,"P(And(X>=8,X<=12))",$P{\left(X \geq 8 \wedge X \leq 12 \right)}=0....,La duración media de un televisor es de 10 año...,1BC,Parcial 3ª Evaluación,ex21e05-0,5,1,1,False,False


FileNotFoundError: [Errno 2] No such file or directory: '/home/hp/mi_git/mat1bac_cit/notebooks/parcial 3.pdf' -> '../ejercicios/build/parcial 3.pdf'